In [ ]:
import requests
from pprint import pprint
from bs4 import BeautifulSoup
from pymongo import MongoClient
MONGO_HOSTNAME = 'localhost'
MONGO_PORT = 27017
import json


In [161]:
def get_coins(url):
    """Get coin list from URL

    Args:
        url (string): sandwich.finance URL to fetch

    Return:
        string: text file contained on page
    """
    res = requests.get(url)
    if res.ok:
        return [coin for coin in res.content.decode("utf-8").split("\n")]
    


In [144]:
urls_to_add = []
def get_sandwich_data():
    """Scrape sandwich.finance to get the market URLs for all exchanges

    Returns:
        list: List of objects containing the exchange, market name, and url
    """
    base_url = "https://www.sandwich.finance/"
    res = requests.get(base_url)
    soup = BeautifulSoup(res.content, 'html.parser')

    table = soup.select_one(
        "#__next > div > main > section.section.lists > div > div > div > div")


    sandwich_data = []
    markets_table = table.select("ul")
    exchange_list = [e.text.lower() for e in table.select("h2")]
    if len(markets_table) == len(exchange_list):
        for markets, exch in zip(markets_table, exchange_list):
            for market in markets.select("li"):
                a_tag = market.find('a')
                link = a_tag['href']
                market_name = a_tag['title'].split(
                    "-")[1].strip().replace(" ", "_").lower()
                sandwich_data.append(
                    {'exchange': exch, 'market': market_name, 'url': link})

    return sandwich_data

sandwich_data = get_sandwich_data()

sandwich_data


[{'exchange': 'binance',
  'market': 'all_markets',
  'url': 'https://sandwichfinance.blob.core.windows.net/files/binance_all_markets.txt'},
 {'exchange': 'binance',
  'market': 'btc_markets',
  'url': 'https://sandwichfinance.blob.core.windows.net/files/binance_btc_markets.txt'},
 {'exchange': 'binance',
  'market': 'eth_markets',
  'url': 'https://sandwichfinance.blob.core.windows.net/files/binance_eth_markets.txt'},
 {'exchange': 'binance',
  'market': 'usdt_markets',
  'url': 'https://sandwichfinance.blob.core.windows.net/files/binance_usdt_markets.txt'},
 {'exchange': 'binance',
  'market': 'bnb_markets',
  'url': 'https://sandwichfinance.blob.core.windows.net/files/binance_bnb_markets.txt'},
 {'exchange': 'binance',
  'market': 'usdt_perpetual_futures',
  'url': 'https://sandwichfinance.blob.core.windows.net/files/binancefuturesf_usdt_perpetual_futures.txt'},
 {'exchange': 'binance',
  'market': 'coin_perpetual_futures',
  'url': 'https://sandwichfinance.blob.core.windows.net/fil

In [165]:
exchange='coinbase' 
market='usd_markets'
x = None
with MongoClient(MONGO_HOSTNAME, MONGO_PORT) as client:
    url_db = client.sandwich['urls']
    x = url_db.find_one({'exchange': exchange, 'market': market})['url']
    coins_db = client.sandwich[exchange]
    coins_db.insert_one({'market': market, 'coins': get_coins(x)})
x


'https://sandwichfinance.blob.core.windows.net/files/coinbase_usd_markets.txt'

In [179]:
with MongoClient(MONGO_HOSTNAME, MONGO_PORT) as client:
    url_db = client.sandwich['urls']
    old_urls = list(url_db.find({}))
    print(old_urls)

[{'_id': ObjectId('61f628451aed01447f8b5aa0'), 'exchange': 'binance', 'market': 'all_markets', 'url': 'https://sandwichfinance.blob.core.windows.net/files/binance_all_markets.txt'}, {'_id': ObjectId('61f628451aed01447f8b5aa1'), 'exchange': 'binance', 'market': 'btc_markets', 'url': 'https://sandwichfinance.blob.core.windows.net/files/binance_btc_markets.txt'}, {'_id': ObjectId('61f628451aed01447f8b5aa2'), 'exchange': 'binance', 'market': 'eth_markets', 'url': 'https://sandwichfinance.blob.core.windows.net/files/binance_eth_markets.txt'}, {'_id': ObjectId('61f628451aed01447f8b5aa3'), 'exchange': 'binance', 'market': 'usdt_markets', 'url': 'https://sandwichfinance.blob.core.windows.net/files/binance_usdt_markets.txt'}, {'_id': ObjectId('61f628451aed01447f8b5aa4'), 'exchange': 'binance', 'market': 'bnb_markets', 'url': 'https://sandwichfinance.blob.core.windows.net/files/binance_bnb_markets.txt'}, {'_id': ObjectId('61f628451aed01447f8b5aa5'), 'exchange': 'binance', 'market': 'usdt_perpetu

In [164]:
with MongoClient(MONGO_HOSTNAME, MONGO_PORT) as client:
    coins_db = client.sandwich[exchange]
    coins_db.delete_one({"market": market})


In [168]:
with MongoClient(MONGO_HOSTNAME, MONGO_PORT) as client:
    url_db = client.sandwich['urls']
    coins_db = client.sandwich[exchange]

    url = url_db.find_one({'exchange': exchange, 'market': market})['url']

    old_coins = coins_db.find_one({'market': market})['coins']
    new_coins = get_coins(url)
    
    print(len(old_coins))
    print(len(new_coins))
    
    if len(new_coins) != len(old_coins):
        print("here")
        # Only perform if coins have been added/removed
        coins_db.delete_one({"market": market})
        coins_db.insert_one({'market': market, 'coins': new_coins})

150
149
here


In [173]:
def fetch_exchanges():
    with MongoClient(MONGO_HOSTNAME, MONGO_PORT) as client:
        url_db = client.sandwich['urls']
        return [e for e in url_db.find().distinct('exchange')]
def fetch_markets(exchange):
    if exchange:
        with MongoClient(MONGO_HOSTNAME, MONGO_PORT) as client:
            url_db = client.sandwich['urls']
            return [m['market'] for m in url_db.find({'exchange': exchange})]
        
def fetch_url(exchange, market):
    if exchange and market:
        with MongoClient(MONGO_HOSTNAME, MONGO_PORT) as client:
            url_db = client.sandwich['urls']
            return url_db.find_one({'exchange': exchange, 'market': market})['url']
# fetch_url('coinbase','usd_markets')
fetch_markets('coinbase')

['all_markets',
 'btc_markets',
 'usd_markets',
 'usdc_markets',
 'eur_markets',
 'gbp_markets']

In [150]:
with MongoClient(MONGO_HOSTNAME, MONGO_PORT) as client:
        url_db = client.sandwich['urls']
        urls = [e for e in url_db.find().distinct('url')]
        print(urls)

['https://sandwichfinance.blob.core.windows.net/files/binance_all_markets.txt', 'https://sandwichfinance.blob.core.windows.net/files/binance_bnb_markets.txt', 'https://sandwichfinance.blob.core.windows.net/files/binance_btc_markets.txt', 'https://sandwichfinance.blob.core.windows.net/files/binance_eth_markets.txt', 'https://sandwichfinance.blob.core.windows.net/files/binance_usdt_markets.txt', 'https://sandwichfinance.blob.core.windows.net/files/binancefuturesd_coin_perpetual_futures.txt', 'https://sandwichfinance.blob.core.windows.net/files/binancefuturesf_usdt_perpetual_futures.txt', 'https://sandwichfinance.blob.core.windows.net/files/bitfinex_all_markets.txt', 'https://sandwichfinance.blob.core.windows.net/files/bitfinex_btc_markets.txt', 'https://sandwichfinance.blob.core.windows.net/files/bitfinex_derivatives.txt', 'https://sandwichfinance.blob.core.windows.net/files/bitfinex_eth_markets.txt', 'https://sandwichfinance.blob.core.windows.net/files/bitfinex_usd_markets.txt', 'https:

In [145]:
def update_urls():
    # Run this on timer
    # If I have to update
    with MongoClient(MONGO_HOSTNAME, MONGO_PORT) as client:
        db = client.sandwich
        db['urls'].insert_many(sandwich_data)
        
update_urls()

In [ ]:
urls = []
with MongoClient(MONGO_HOSTNAME, MONGO_PORT) as client:
        url_db = client.sandwich['urls']
        urls = [e for e in url_db.find().distinct('url')]
        
        for u in urls:
            print(url_db.find_one({'url':u})['exchange'])


In [ ]:
exchange = 'coinbase'
with MongoClient(MONGO_HOSTNAME, MONGO_PORT) as client:
        url_db = client.sandwich['urls']
        markets = [m['url'] for m in url_db.find(
            {'exchange': exchange, 'market': 'usd_markets'})]
        print(markets)
        